In [1]:
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import os
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj,
                                   write_linkage_ribbon_output_florin)

from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods
import vis.fields
import matplotlib.cm as cm
import time

In [2]:
import parallelism
parallelism.set_max_num_tbb_threads(12)
parallelism.set_hessian_assembly_num_threads(4)
parallelism.set_gradient_assembly_num_threads(4)

In [3]:
# Set to True in the options below if you already optimized a specific linkage
# and you would like to reuse the optimized linkage. Loading is performed in
# place of the full design optimization.
# NOTE: Doesn't seem to work at the moment, leave at False
LOAD_OPTIMIZED_DOFS = False

# Set to False if strips cannot be labeled into families for a model
USE_FAMILY_LABEL = False

In [4]:
rod_length = 0.3534025445286393
width = rod_length / 15 * 5
thickness = width / 5 * 0.35

In [5]:
print(width, thickness)

0.11780084817621309 0.008246059372334916


In [6]:
# Sphere 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [thickness, width]
MODEL_NAME = "sphere_1"
MODEL_PATH = osp.join(weaving_dir + 'normalized_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'normalized_objs/surface_models/{}.obj'.format(MODEL_NAME))
RIBBON_NAME = "{}_strip".format(MODEL_NAME)

In [7]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1;
OPTS.beta = 1e-8
OPTS.niter = 200
OPTS.verboseNonPosDef = False

In [8]:
rw = 0.01
sw = 0.01

In [9]:
def initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION, model_path = MODEL_PATH, use_constant_width = True, width_scale = [0.5, 2], interleaving_type = InterleavingType.triaxialWeave):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, useCenterline, model_path, subdivision_res, False, interleaving_type)
    if use_constant_width:
        l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    else:
        cross_section_scaling.apply_density_based_cross_sections(l, elastic_rods.CrossSection.construct('rectangle', 2000, 0.3, [RIBBON_CS[0], width_scale[0] * RIBBON_CS[1]]), elastic_rods.CrossSection.construct('rectangle', 2000, 0.3, [RIBBON_CS[0], width_scale[1] * RIBBON_CS[1]]))

    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

### With equilibrium exception

In [10]:
with so(): curved_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH, useCenterline = True, model_path = MODEL_PATH, cross_section = RIBBON_CS, subdivision_res = SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [11]:
# curved_linkage.setDoFs(curved_linkage.getDoFs())

In [12]:
print(curved_linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Attraction))
print(curved_linkage.energy(elastic_rods.SurfaceAttractionEnergyType.Elastic))
print(curved_linkage.energyBend())
print(curved_linkage.energyTwist())
print(curved_linkage.energyStretch())

0.0001790780256093674
0.24153433955530332
0.21254296048589566
1.6895515537588205e-05
0.028974483553870015


In [13]:
def curved_callback(prob, i):
    curved_linkage_view.update()
    return

In [14]:
OPTS.niter = 1000

In [15]:
elastic_rods.compute_equilibrium(curved_linkage, callback = curved_callback, options = OPTS)

0	0.241713	31.4001	31.4001	1	1
1	0.0541772	2.22707	2.22707	1	1
2	0.0357471	0.817509	0.817509	1	1
3	0.0256738	0.46611	0.46611	1	1
4	0.0184431	0.372737	0.372737	1	1
5	0.013537	0.268692	0.268692	1	1
6	0.0102447	0.217106	0.217106	1	1
7	0.00794302	0.289826	0.289826	1	1
8	0.00630224	0.259164	0.259164	1	1
9	0.00500034	0.234324	0.234324	1	1
10	0.00408817	0.238731	0.238731	1	1
11	0.00346313	0.259458	0.259458	1	1
12	0.00283301	0.687844	0.687844	0.015625	1
13	0.00264329	0.405149	0.405149	1	1
14	0.00242874	0.094137	0.094137	1	1
15	0.00235463	0.0243524	0.0243524	1	1
16	0.00222385	0.00485502	0.00485502	1	1
17	0.00198992	0.00844676	0.00844676	0.25	0
18	0.00155008	0.0825429	0.0825429	0.5	0
19	0.00132098	0.0740405	0.0740405	1	0
20	0.00122325	0.128989	0.128989	1	1
21	0.0011763	0.0357092	0.0357092	1	0
22	0.00116599	0.0945287	0.0945287	0.125	1
23	0.00116401	0.0408935	0.0408935	1	0
24	0.00116378	0.0960585	0.0960585	0.5	1
25	0.0011636	0.0424828	0.0424828	1	0
26	0.00116312	0.0355377	0.0355377	0.5	0
27	0.0011

In [16]:
# with so(): elastic_rods.compute_equilibrium(curved_linkage)

In [17]:
print(curved_linkage.energy())
print(curved_linkage.energyBend())
print(curved_linkage.energyTwist())
print(curved_linkage.energyStretch())

0.0011613047500400658
0.0006018058252078156
0.0003018298460863259
1.8598855581731607e-06


In [18]:
print(curved_linkage.energy())
print(curved_linkage.energyBend())
print(curved_linkage.energyTwist())
print(curved_linkage.energyStretch())

0.0011613047500400658
0.0006018058252078156
0.0003018298460863259
1.8598855581731607e-06


In [19]:
curved_dpo = elastic_rods.get_designParameter_optimizer(curved_linkage, rw, sw, callback=curved_callback)
curved_dpo.options.niter = 300
prev_time_stamp = time.time()
curved_cr = curved_dpo.optimize()

0	1.01	6.19084	6.19084	0.0625	0
1	0.922213	106.899	106.899	1	0
2	0.696089	503.775	503.775	0.125	1
3	0.684641	436.808	436.808	1	1
4	0.577497	173.693	173.693	1	1
5	0.564079	7.22494	7.22494	1	1
6	0.559046	7.84322	7.84322	1	1
7	0.554396	5.34818	5.34818	1	1
8	0.55035	5.47696	5.47696	1	1
9	0.547292	5.52373	5.52373	1	1
10	0.545436	5.2989	5.2989	1	1
11	0.544652	5.0619	5.0619	1	1
12	0.544336	4.96424	4.96424	1	1
13	0.543963	4.90506	4.90506	1	1
14	0.543264	4.79586	4.79586	1	1
15	0.541968	4.59737	4.59737	1	1
16	0.539699	4.26582	4.26582	1	1
17	0.536064	3.76331	3.76331	1	1
18	0.530965	3.07183	3.07183	1	1
19	0.525199	2.27948	2.27948	1	1
20	0.520224	1.77602	1.77602	1	1
21	0.515524	1.83441	1.83441	1	1
22	0.508537	3.05404	3.05404	1	1
23	0.497692	7.60191	7.60191	0.25	0
24	0.489175	33.7374	33.7374	1	0
25	0.471412	96.5494	96.5494	1	0
26	0.467419	17.5594	17.5594	1	0
27	0.465914	8.91827	8.91827	0.125	0
28	0.465811	9.08385	9.08385	1	0
29	0.464891	1.37359	1.37359	0.015625	0
30	0.464886	1.45903	1.45903	0.25	0
3

In [48]:
print(curved_linkage.energy())
print(curved_linkage.energyBend())
print(curved_linkage.energyTwist())
print(curved_linkage.energyStretch())

0.0004034844321390627
0.0003643047308071722
9.389661662522952e-07
1.742028970959109e-08
